Goals:
1. Create a corpus of all yt video descriptions for scam videos in the data.
2. vectorize this.
3. Use this to create a regressor capturing the degree to which a video description is "spam-speaky" according to ML result (or dichotomous 1-spammy, 0-notSpammy).
4.Use this bag-of-words model to generate values for a new variable, "spam-speakiness", for every observation in the dataset .
5. (optional) repeat this process with a dataset of spam-lookalike videos which warn AGAINST scams and spam and attempt to create another regressor, "virtue-speakiness".
6. get "has_numbers" and "number_sum" for numbers in character vars
7. multiply-impute missing values b4 machine learning (training data only, drop missing on test!!!!)
8. get capital letter frequency
9. get emoji frequency

In [1]:
import os
import json 
import pandas as pd
import pymongo
#from google.colab import drive
#nltk.download('stopwords')
from pymongo import MongoClient
import socket
import urllib.request as urllib2
import pandas as pd
import numpy as np
import nltk  
import numpy as np  
import random  
import string

import urllib.request  
import re  
from string import punctuation
from os import listdir
from collections import Counter
from nltk.corpus import stopwords
import math





In [2]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)


db = client.videos_mdb
collection = db["videos"]  

In [3]:
#pull data from mongo (just one iv for now)
linVarList=["id","spam","description"]
df=pd.DataFrame(columns=linVarList)
row=[]
counter=0
data = collection.find({})
for item in data:
    for var in linVarList:
        if var in item:
            row.append(item[var])
        else:
            row.append(np.nan)
    df.loc[counter] = row
    counter+=1
    row=[]

In [4]:
spam=df.loc[df["spam"]==1]

In [19]:
#Create training data from X percent of the total spam entries
XPercent=math.floor(len(spam)*0.2)
xtrain = spam.sample(n=XPercent, random_state = 42)

In [20]:
#Get the rest of the data as testing data
xtest=pd.concat([spam, xtrain]).drop_duplicates(keep=False)

In [21]:
rawDescriptions=[(item for item in xtrain["description"])]
corpus=""
for item in rawDescriptions:
    corpus=corpus.join(item)

In [22]:
 #nltk.download('punkt')

In [23]:
from emoji import UNICODE_EMOJI

def is_emoji(s):
    count = 0
    for emoji in UNICODE_EMOJI:
        count += s.count(emoji)
        if count > 1:
            return False
    return bool(count)

In [24]:
from nltk.tokenize.casual import TweetTokenizer
import enchant
import re
t = TweetTokenizer()

In [25]:

#TO DO: exclude if is not a word OR is emoji OR is short boring word

# turn a doc into clean tokens
def clean_doc(doc):
	# split into tokens 
	tokens=t.tokenize(doc)
	# remove punctuation from each token
	table = str.maketrans('', '', string.punctuation)
	tokens = [w.translate(table) for w in tokens]
	# remove remaining tokens that are not alphabetic
	#tokens = [word for word in tokens if word.isalpha() or is_emoji(word)]
	# filter out stop words
	stop_words = set(stopwords.words('english'))
	tokens = [w for w in tokens if not w in stop_words]
	# filter out short tokens
	#tokens = [word for word in tokens if len(word) > 1]
	return tokens
	

# load the document
tokens = clean_doc(corpus)

In [26]:
wordfreq = {}


for token in tokens:
    if token not in wordfreq.keys():
        wordfreq[token.lower()] = 1
    else:
        wordfreq[token.lower()] += 1

exclaim=0
question=0
hashy=0
atme=0

for item in corpus:
    if item=="!":
        exclaim+=1
    elif item == "?":
        question+=1
    elif item =="#":
        hashy+=1


wordfreq["?"]=question
wordfreq["!"]=exclaim



In [27]:
punctuation = [",","'",".",";",":","(",")",""]
wordfreq.pop("", None)
for item in punctuation:
    wordfreq.pop(item, None)

In [28]:
import heapq
#most_freq = [i for i in wordfreq if wordfreq[i] >= 1 and i[0:4]!="http" ]
#wordfreqmost_freq = heapq.nlargest(200, wordfreq, key=wordfreq.get)

In [29]:

sorted_freq=sorted(wordfreq.items(), key=lambda x: x[1], reverse=True)
most_freq=sorted_freq[:200]

In [30]:
most_freq

[('!', 97),
 ('▬', 90),
 ('?', 55),
 ('👉', 38),
 ('✅', 38),
 ('🔥', 37),
 ('➡', 37),
 ('️', 37),
 ('video', 24),
 ('1', 21),
 ('always', 18),
 ('experience', 17),
 ('cash', 17),
 ('★', 16),
 ('’', 16),
 ('►', 15),
 ('➜', 14),
 ('💥', 12),
 ('mentioned', 12),
 ('financial', 12),
 ('information', 12),
 ('⚫', 12),
 ('2021', 12),
 ('200', 12),
 ('—', 12),
 ('🔴', 10),
 ('educational', 10),
 ('purposes', 10),
 ('guarantee', 10),
 ('500', 10),
 ('3', 9),
 ('🔔', 9),
 ('risk', 9),
 ('100', 9),
 ('300', 9),
 ('need', 9),
 ('home', 9),
 ('typical', 8),
 ('level', 8),
 ('2', 8),
 ('showing', 8),
 ('5', 7),
 ('ways', 7),
 ('👍', 7),
 ('advisor', 7),
 ('advice', 7),
 ('50', 7),
 ('achieve', 7),
 ('videos', 6),
 ('👇', 6),
 ('results', 6),
 ('personal', 6),
 ('research', 6),
 ('steps', 6),
 ('description', 6),
 ('product', 6),
 ('receive', 6),
 ('income', 6),
 ('earning', 6),
 ('400', 6),
 ('examples', 6),
 ('🤑', 6),
 ('💰', 6),
 ('products', 6),
 ('◾', 6),
 ('techniques', 5),
 ('say', 5),
 ('seen', 5),
 

In [14]:
boringWords=["with","that","this","will","your","there", "wesleyvirgin", "wesleymilliondollarvirgin","just", "they", "much","kaise","then", "than", "when", "here","these", "girl", "what","where","from", "wesley", "virgin", "also",  "about", "them", "which", "dave", "very"] #"girl" is removed b/c comes from name of big scam youtuber and leaving it in risks sexist codings

In [ ]:
#number of words in spamwords/ total number of words = regressor